# Importing libraries

In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
# Install required dependencies
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 15.0 MB/s eta 0:00:00


# Unified Model Architecture

Input Image
   → Pretrained YOLO (Feature Extraction + Bounding Box Predictions)
      → ROI Pooling Layer (Extract Features for Each Bounding Box)
         → Classifier Head (Fine-tuned VGG16 or Custom CNN for Defect Classification)
            → Combined Output (Bounding Boxes + Defect Labels)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics tensorflow keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.9/932.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

# Loding dataset

In [ ]:
import zipfile

# Path to the uploaded file
zip_file_path = '/content/train.zip'

# Path to extract the contents
extract_path = '/content/train'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/train


In [ ]:
import zipfile

# Path to the uploaded file
zip_file_path = '/content/test.zip'

# Path to extract the contents
extract_path = '/content/test'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/test


In [ ]:
import os
import tensorflow as tf

# Dataset loader with robustness
def load_dataset(image_dir, label_dir, target_size=(640, 640)):
    images = []
    bboxes = []
    labels = []
    missing_labels = []
    corrupt_labels = []

    for img_file in os.listdir(image_dir):
        try:
            # Load image
            img_path = os.path.join(image_dir, img_file)
            image = tf.io.read_file(img_path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, target_size) / 255.0  # Normalize to [0, 1]

            # Load corresponding label
            label_file = os.path.join(label_dir, img_file.replace('.jpg', '.txt'))
            if not os.path.exists(label_file):
                missing_labels.append(img_file)
                continue  # Skip if label file is missing

            with open(label_file, 'r') as f:
                label_data = f.readlines()

            bbox = []
            class_ids = []
            for line in label_data:
                try:
                    # Parse YOLO label format
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    class_ids.append(int(class_id))

                    # Convert YOLO format to [x_min, y_min, x_max, y_max]
                    x_min = (x_center - width / 2) * target_size[0]
                    y_min = (y_center - height / 2) * target_size[1]
                    x_max = (x_center + width / 2) * target_size[0]
                    y_max = (y_center + height / 2) * target_size[1]
                    bbox.append([x_min, y_min, x_max, y_max])
                except ValueError:
                    corrupt_labels.append(label_file)
                    break  # Skip this image if label parsing fails

            if bbox:  # Add only if bounding boxes are valid
                images.append(image)
                bboxes.append(bbox)
                labels.append(class_ids)
        except Exception as e:
            print(f"Error processing file {img_file}: {e}")

    if missing_labels:
        print(f"Missing labels for {len(missing_labels)} images.")
    if corrupt_labels:
        print(f"Corrupt labels in {len(corrupt_labels)} files.")

    return images, bboxes, labels

# Paths to train/test folders
train_image_dir = '/content/drive/MyDrive/Building_Defects/Dataset/train/images'
train_label_dir = '/content/drive/MyDrive/Building_Defects/Dataset/train/labels'
test_image_dir = '/content/drive/MyDrive/Building_Defects/Dataset/test/images'
test_label_dir = '/content/drive/MyDrive/Building_Defects/Dataset/test/labels'

# Load train and test datasets
train_images, train_bboxes, train_labels = load_dataset(train_image_dir, train_label_dir)
test_images, test_bboxes, test_labels = load_dataset(test_image_dir, test_label_dir)

print(f"Loaded {len(train_images)} training images and {len(test_images)} test images.")


Missing labels for 143 images.
Missing labels for 143 images.
Loaded 573 training images and 37 test images.


In [ ]:
import shutil
import os
from sklearn.model_selection import train_test_split

# Paths for train and test datasets
train_images_path = "/content/drive/MyDrive/Building_Defects/train/images"
train_labels_path = "/content/drive/MyDrive/Building_Defects/train/labels"
test_images_path = "/content/drive/MyDrive/Building_Defect/test/images"
test_labels_path = "/content/drive/MyDrive/Building_Defects/test/labels"

combined_images_path = "/content/drive/MyDrive/Building_Defects/Dataset/combined/images"
combined_labels_path = "/content/drive/MyDrive/Building_Defects/Dataset/combined/labels"
"""
# Paths for train and test datasets
train_images_path = "/content/Dataset/train/images"
train_labels_path = "/content/Dataset/train/labels"
test_images_path = "/content/Dataset/test/images"
test_labels_path = "/content/Dataset/test/labels"

combined_images_path = "/content/Dataset/combined/images"
combined_labels_path = "/content/Dataset/combined/labels"
"""

# Create output directories if they don't exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)

# Combine all image and label files into one list
all_images = os.listdir(combined_images_path)
all_labels = os.listdir(combined_labels_path)

In [ ]:
import os
import shutil

# Define your source and destination paths for train and test datasets
train_images_path = "/content/drive/MyDrive/Building_Defects/Dataset/train/images"
train_labels_path = "/content/drive/MyDrive/Building_Defects/Dataset/train/labels"
test_images_path = "/content/drive/MyDrive/Building_Defects/Dataset/test/images"
test_labels_path = "/content/drive/MyDrive/Building_Defects/Dataset/test/labels"


## Preparing Dataset pipeline

In [ ]:
def create_tf_dataset(images, bboxes, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((images, bboxes, labels))
    dataset = dataset.shuffle(buffer_size=len(images))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create train and test datasets
batch_size = 16
train_dataset = create_tf_dataset(train_images, train_bboxes, train_labels, batch_size)
test_dataset = create_tf_dataset(test_images, test_bboxes, test_labels, batch_size)


In [ ]:
def yolo_loss(pred_bboxes, true_bboxes):
    # Replace this with a custom YOLO loss function
    return tf.reduce_mean(tf.square(pred_bboxes - true_bboxes))

def classifier_loss(pred_labels, true_labels):
    return tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(true_labels, pred_labels)

def combined_loss(bbox_loss, label_loss, alpha=1.0, beta=1.0):
    return alpha * bbox_loss + beta * label_loss


# Define the YOLO + Classifier Model

In [ ]:
import tensorflow as tf
from ultralytics import YOLO
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Load pretrained YOLO model
yolo_model = YOLO('yolov5s.pt')  # Pretrained YOLOv8 weights

def yolo_backbone(image):
    """
    Use YOLO as a feature extractor and bounding box predictor.
    """
    results = yolo_model(image)  # Forward pass through YOLO
    bboxes = results[0].boxes.xyxy  # Extract bounding boxes
    features = results[0].features  # Feature map from YOLO
    return features, bboxes

# Define the Classifier Head (e.g., VGG16 or Custom CNN)
def build_classifier(input_shape=(7, 7, 512), num_classes=4):
    """
    Build the classifier head for defect classification.
    """
    base_model = tf.keras.applications.VGG16(
        weights='imagenet', include_top=False, input_shape=input_shape
    )
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Global pooling for feature reduction
    x = Dense(256, activation='relu')(x)  # Fully connected layer
    output = Dense(num_classes, activation='softmax')(x)  # Output layer
    return Model(inputs=base_model.input, outputs=output)

# Build the Unified Model
class YOLOWithClassifier(tf.keras.Model):
    def __init__(self, num_classes):
        super(YOLOWithClassifier, self).__init__()
        self.yolo = yolo_model
        self.classifier = build_classifier(num_classes=num_classes)

    def call(self, inputs):
        # YOLO for feature extraction and bounding box prediction
        features, bboxes = yolo_backbone(inputs)

        # ROI pooling: Use bounding boxes to extract features
        rois = self.roi_pooling(features, bboxes)

        # Classification head
        predictions = self.classifier(rois)

        return bboxes, predictions

    def roi_pooling(self, features, bboxes):
        """
        ROI pooling implementation for extracting bounding box features.
        """
        pooled_features = []
        for bbox in bboxes:
            x1, y1, x2, y2 = map(int, bbox)
            roi = features[:, y1:y2, x1:x2, :]  # Crop the ROI
            roi = tf.image.resize(roi, (7, 7))  # Resize to match classifier input
            pooled_features.append(roi)
        return tf.stack(pooled_features)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 24.7MB/s]


In [ ]:
def yolo_loss(pred_bboxes, true_bboxes):
    """
    Compute YOLO bounding box loss (e.g., MSE for regression).
    """
    return tf.reduce_mean(tf.square(pred_bboxes - true_bboxes))

def classifier_loss(pred_labels, true_labels):
    """
    Compute classification loss (e.g., Categorical Crossentropy).
    """
    return tf.keras.losses.categorical_crossentropy(true_labels, pred_labels)

def combined_loss(yolo_loss_value, classifier_loss_value, alpha=1.0, beta=1.0):
    """
    Combine YOLO and classifier loss with weights alpha and beta.
    """
    return alpha * yolo_loss_value + beta * classifier_loss_value


In [ ]:
# Paths
# Define your pathsPorosity Overweld
categories = [ 'Porosity','Overweld','Underfilled', 'Undercut']
label_map = {0: 'Porosity', 1: 'Overweld', 2: 'Underfilled', 3: 'Undercut'}

In [ ]:
import zipfile

# Path to the uploaded file
zip_file_path = '/content/drive/MyDrive/Dataset-20250102T092628Z-001.zip'

# Path to extract the contents
extract_path = '/content'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")


Files extracted to /content


Files extracted to /content/train


In [ ]:
import zipfile

# Path to the uploaded file
zip_file_path = '/content/test.zip'

# Path to extract the contents
extract_path = '/content/test'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/test


In [ ]:
"""
# Now you can download the zip files
from google.colab import files
files.download('/content/augmented_images_labels.zip')
files.download('/content/augmented_labels.zip')"""

"\n# Now you can download the zip files\nfrom google.colab import files\nfiles.download('/content/augmented_images_labels.zip')\nfiles.download('/content/augmented_labels.zip')"

In [ ]:
import cv2
import os
import albumentations as A

# Paths
input_images_path = "/content/drive/MyDrive/Defects_welding/yolo/original/images"
input_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/original/labels"
output_images_path = "/content/drive/MyDrive/Defects_welding/yolo/Agu/images"
output_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/Agu/labels"

# Create output directories if they don't exist
os.makedirs(output_images_path, exist_ok=True)
os.makedirs(output_labels_path, exist_ok=True)

# Augmentation pipeline
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=15, p=0.4),
        A.RandomBrightnessContrast(p=0.4),
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
)

# Target number of images for each category
target_images_per_category = {
    0: 500,  # overweld
    1: 500,  # porosity
    2: 500,  # undercut
    3: 500,  # underfilled
}

# List all image files in the images folder
image_files = [f for f in os.listdir(input_images_path) if f.endswith('.jpg')]

# Function to get the category of an image from its label file
def get_category_from_label(label_path):
    with open(label_path, "r") as f:
        class_id = int(f.readline().split()[0])  # First entry in label corresponds to the class
    return class_id

# Organize images by category
images_by_category = {0: [], 1: [], 2: [], 3: []}
for img_file in image_files:
    img_path = os.path.join(input_images_path, img_file)
    label_path = os.path.join(input_labels_path, img_file.replace(".jpg", ".txt"))

    # Get the category for the image
    category = get_category_from_label(label_path)

    # Add the image to the appropriate category list
    images_by_category[category].append(img_file)

# Process each category and augment images to meet the target
for category, image_list in images_by_category.items():
    category_name = categories[category]
    current_image_count = len(image_list)
    target_count = target_images_per_category[category]
    augmentations_needed = target_count - current_image_count

    print(f"Processing category: {category_name} | Current: {current_image_count}, Target: {target_count}, Needed: {augmentations_needed}")

    if augmentations_needed > 0:
        for img_file in image_list:
            img_path = os.path.join(input_images_path, img_file)
            image = cv2.imread(img_path)

            label_path = os.path.join(input_labels_path, img_file.replace(".jpg", ".txt"))
            with open(label_path, "r") as f:
                bboxes = []
                class_labels = []
                for line in f.readlines():
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:])
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

            # Generate the required number of augmentations
            for i in range(augmentations_needed):
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                augmented_image = augmented["image"]
                augmented_bboxes = augmented["bboxes"]

                # Save augmented image
                aug_img_name = f"aug_{category}_{i}_{img_file}"
                cv2.imwrite(os.path.join(output_images_path, aug_img_name), augmented_image)

                # Save augmented labels
                aug_label_name = f"aug_{category}_{i}_{img_file.replace('.jpg', '.txt')}"
                with open(os.path.join(output_labels_path, aug_label_name), "w") as f:
                    for bbox, class_id in zip(augmented_bboxes, class_labels):
                        f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

                augmentations_needed -= 1
                if augmentations_needed == 0:
                    break
    else:
        print(f"Category '{category_name}' already meets or exceeds the target count. No augmentation needed.")

print("Augmentation completed!")


In [ ]:
import cv2
import os
import albumentations as A
from matplotlib import pyplot as plt
categories = ['Porosity','Overweld', 'Underfilled', 'Undercut']
label_map = {0: 'Porosity', 1: 'Overweld', 2: 'Underfilled', 3: 'Undercut'}
# Paths
input_images_path = "/content/drive/MyDrive/Defects_welding/yolo/original/images"
input_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/original/labels"
output_images_path = "/content/drive/MyDrive/Defects_welding/yolo/Testdata/images"
output_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/Testdata/labels"

# Create output directories if they don't exist
os.makedirs(output_images_path, exist_ok=True)
os.makedirs(output_labels_path, exist_ok=True)

# Augmentation pipeline
transform = A.Compose(
    [

        # **Additional Augmentations**
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),  # Add Gaussian noise
        A.MotionBlur(blur_limit=5, p=0.3),  # Apply motion blur  # Random affine transformations
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.3),  # Contrast Limited Adaptive Histogram Equalization
        A.ToGray(p=0.2),  # Convert image to grayscale with probability  # Simulate occlusions
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
)

# Target number of images for each category
target_images_per_category = {
    0: 100,  # overweld
    1: 100,  # porosity
    2: 100,  # undercut
    3: 100,  # underfilled
}

# List all image files in the images folder
image_files = [f for f in os.listdir(input_images_path) if f.endswith('.jpg')]

# Function to get the category of an image from its label file
def get_category_from_label(label_path):
    with open(label_path, "r") as f:
        class_id = int(f.readline().split()[0])  # First entry in label corresponds to the class
    return class_id

# Organize images by category
images_by_category = {0: [], 1: [], 2: [], 3: []}
for img_file in image_files:
    img_path = os.path.join(input_images_path, img_file)
    label_path = os.path.join(input_labels_path, img_file.replace(".jpg", ".txt"))

    # Get the category for the image
    category = get_category_from_label(label_path)

    # Add the image to the appropriate category list
    images_by_category[category].append(img_file)

# Process each category and augment images to meet the target
for category, image_list in images_by_category.items():
    category_name = categories[category]
    current_image_count = 0  # chaned code
    target_count = target_images_per_category[category]
    augmentations_needed = target_count - current_image_count

    print(f"Processing category: {category_name} | Current: {current_image_count}, Target: {target_count}, Needed: {augmentations_needed}")

    if augmentations_needed > 0:
        for img_file in image_list:
            img_path = os.path.join(input_images_path, img_file)
            image = cv2.imread(img_path)

            label_path = os.path.join(input_labels_path, img_file.replace(".jpg", ".txt"))
            with open(label_path, "r") as f:
                bboxes = []
                class_labels = []
                for line in f.readlines():
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:])
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)

            # Generate the required number of augmentations
            for i in range(augmentations_needed):
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                augmented_image = augmented["image"]
                augmented_bboxes = augmented["bboxes"]

                # Save augmented image
                aug_img_name = f"aug_{category}_{i}_{img_file}"
                cv2.imwrite(os.path.join(output_images_path, aug_img_name), augmented_image)

                # Save augmented labels
                aug_label_name = f"aug_{category}_{i}_{img_file.replace('.jpg', '.txt')}"
                with open(os.path.join(output_labels_path, aug_label_name), "w") as f:
                    for bbox, class_id in zip(augmented_bboxes, class_labels):
                        f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

                augmentations_needed -= 1
                if augmentations_needed == 0:
                    break
    else:
        print(f"Category '{category_name}' already meets or exceeds the target count. No augmentation needed.")

print("Augmentation completed!")

<ipython-input-3-4b48f50a99dd>:22: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),  # Add Gaussian noise


Processing category: Porosity | Current: 0, Target: 100, Needed: 100
Processing category: Overweld | Current: 0, Target: 100, Needed: 100
Processing category: Underfilled | Current: 0, Target: 100, Needed: 100
Processing category: Undercut | Current: 0, Target: 100, Needed: 100
Augmentation completed!


In [ ]:
# Function to count images in each category
def count_images_by_category(images_by_category, input_labels_path):
    category_counts = {0: 0, 1: 0, 2: 0, 3: 0}
    for category, image_list in images_by_category.items():
        category_counts[category] = len(image_list)
    return category_counts

# Print initial counts
initial_counts = count_images_by_category(images_by_category, input_labels_path)
print("Initial Counts Per Category:")
for category, count in initial_counts.items():
    print(f"Category {categories[category]} ({category}): {count} images")

# Augmentation process (no changes to previous code here)

# Count images again after augmentation
final_images_by_category = {0: [], 1: [], 2: [], 3: []}
output_image_files = [f for f in os.listdir(output_images_path) if f.endswith('.jpg')]

for img_file in output_image_files:
    label_path = os.path.join(output_labels_path, img_file.replace(".jpg", ".txt"))
    category = get_category_from_label(label_path)
    final_images_by_category[category].append(img_file)

final_counts = count_images_by_category(final_images_by_category, output_labels_path)
print("\nFinal Counts Per Category After Augmentation:")
for category, count in final_counts.items():
    print(f"Category {categories[category]} ({category}): {count} images")


Initial Counts Per Category:
Category Porosity (0): 32 images
Category Overweld (1): 41 images
Category Underfilled (2): 45 images
Category Undercut (3): 45 images

Final Counts Per Category After Augmentation:
Category Porosity (0): 100 images
Category Overweld (1): 100 images
Category Underfilled (2): 100 images
Category Undercut (3): 100 images


In [ ]:
# Function to count total images in all categories
def count_total_images(category_counts):
    return sum(category_counts.values())

# Print initial counts
initial_counts = count_images_by_category(images_by_category, input_labels_path)
initial_total = count_total_images(initial_counts)

print("Initial Counts Per Category:")
for category, count in initial_counts.items():
    print(f"Category {categories[category]} ({category}): {count} images")
print(f"Total images before augmentation: {initial_total}")

# Augmentation process (no changes to previous code here)

# Count images again after augmentation
final_images_by_category = {0: [], 1: [], 2: [], 3: []}
output_image_files = [f for f in os.listdir(output_images_path) if f.endswith('.jpg')]

for img_file in output_image_files:
    label_path = os.path.join(output_labels_path, img_file.replace(".jpg", ".txt"))
    category = get_category_from_label(label_path)
    final_images_by_category[category].append(img_file)

final_counts = count_images_by_category(final_images_by_category, output_labels_path)
final_total = count_total_images(final_counts)

print("\nFinal Counts Per Category After Augmentation:")
for category, count in final_counts.items():
    print(f"Category {categories[category]} ({category}): {count} images")
print(f"Total images after augmentation: {final_total}")


Initial Counts Per Category:
Category Porosity (0): 32 images
Category Overweld (1): 41 images
Category Underfilled (2): 45 images
Category Undercut (3): 45 images
Total images before augmentation: 163

Final Counts Per Category After Augmentation:
Category Porosity (0): 100 images
Category Overweld (1): 100 images
Category Underfilled (2): 100 images
Category Undercut (3): 100 images
Total images after augmentation: 400


In [ ]:
import os
import shutil
original_images_path = "/content/drive/MyDrive/Defects_welding/yolo/original/images"
original_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/original/labels"
augmented_images_path = "/content/drive/MyDrive/Defects_welding/yolo/Agu/images"
augmented_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/Agu/labels"
"""
original_images_path =  "/content/drive/MyDrive/Building_Defects/Dataset/images"
original_labels_path = "/content/drive/MyDrive/Building_Defects/Dataset/labels"
augmented_images_path = "/content/drive/MyDrive/Building_Defects/Dataset/Aaugment_images"
augmented_labels_path = "/content/drive/MyDrive/Building_Defects/Dataset/Aaugmented_labels"
"""
combined_images_path = "/content/drive/MyDrive/Defects_welding/yolo/combined/images"
combined_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/combined/labels"

# Create output directories if they don't exist
os.makedirs(combined_images_path, exist_ok=True)
os.makedirs(combined_labels_path, exist_ok=True)

# Function to copy images and labels to the combined folder with a label (original/augmented)
def copy_images_and_labels(images_path, labels_path, destination_images_path, destination_labels_path, label_type):
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]  # Adjust image extension if needed

    for image_file in image_files:
        # Copy image file to destination
        src_image_path = os.path.join(images_path, image_file)
        dst_image_path = os.path.join(destination_images_path, f"{label_type}_{image_file}")
        shutil.copy(src_image_path, dst_image_path)

        # Copy corresponding label file to destination
        label_file = image_file.replace(".jpg", ".txt")  # Assuming .txt for label files
        src_label_path = os.path.join(labels_path, label_file)
        dst_label_path = os.path.join(destination_labels_path, f"{label_type}_{label_file}")
        shutil.copy(src_label_path, dst_label_path)

# Copy original images and labels with "original" label
copy_images_and_labels(original_images_path, original_labels_path, combined_images_path, combined_labels_path, "original")

# Copy augmented images and labels with "augmented" label
copy_images_and_labels(augmented_images_path, augmented_labels_path, combined_images_path, combined_labels_path, "augmented")

# Print the number of combined images
combined_image_files = [f for f in os.listdir(combined_images_path) if f.endswith('.jpg')]
print(f"Total images in the combined folder: {len(combined_image_files)}")


Total images in the combined folder: 2000


In [ ]:
# Function to calculate the combined total of original and augmented images
def calculate_totals(images_by_category, augmented_labels_path):
    # Initialize a dictionary to store totals for each category
    combined_counts = {0: 0, 1: 0, 2: 0, 3: 0}

    # Count original images
    for category, files in images_by_category.items():
        combined_counts[category] += len(files)

    # Count augmented images
    augmented_label_files = [f for f in os.listdir(augmented_labels_path) if f.endswith('.txt')]
    for label_file in augmented_label_files:
        label_path = os.path.join(augmented_labels_path, label_file)
        category = get_category_from_label(label_path)
        combined_counts[category] += 1  # Add 1 for each augmented label in the category

    return combined_counts

# Calculate combined totals
combined_counts = calculate_totals(images_by_category, output_labels_path)

# Display combined totals and grand total
print("Combined (Original + Augmented) Counts Per Category:")
grand_total = 0
for category, count in combined_counts.items():
    print(f"Category {categories[category]} ({category}): {count} images")
    grand_total += count

print(f"Grand Total (All Categories): {grand_total} images")

Combined (Original + Augmented) Counts Per Category:
Category Porosity (0): 500 images
Category Overweld (1): 500 images
Category Underfilled (2): 500 images
Category Undercut (3): 500 images
Grand Total (All Categories): 2000 images


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
combined_images_path = "/content/drive/MyDrive/Defects_welding/yolo/combined/images"
combined_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/combined/labels"

train_images_path = "/content/drive/MyDrive/Defects_welding/yolo/train/images"
train_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/train/labels"
test_images_path = "/content/drive/MyDrive/Defects_welding/yolo/test/images"
test_labels_path = "/content/drive/MyDrive/Defects_welding/yolo/test/labels"

# Ensure destination directories exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)

# Get list of image and label files
image_files = sorted(os.listdir(combined_images_path))
label_files = sorted(os.listdir(combined_labels_path))

# Ensure each image has a corresponding label
image_files = [img for img in image_files if os.path.splitext(img)[0] + ".txt" in label_files]
label_files = [lbl for lbl in label_files if os.path.splitext(lbl)[0] + ".jpg" in image_files]

# Perform an 80-20 split
train_images, test_images, train_labels, test_labels = train_test_split(
    image_files, label_files, test_size=0.2, random_state=42
)

# Function to move files
def move_files(file_list, source_folder, destination_folder):
    for file_name in file_list:
        shutil.copy(os.path.join(source_folder, file_name), os.path.join(destination_folder, file_name))

# Move training data
move_files(train_images, combined_images_path, train_images_path)
move_files(train_labels, combined_labels_path, train_labels_path)

# Move testing data
move_files(test_images, combined_images_path, test_images_path)
move_files(test_labels, combined_labels_path, test_labels_path)

print("Dataset successfully split and saved!")

Dataset successfully split and saved!
